In [8]:
%pip install bokeh

Note: you may need to restart the kernel to use updated packages.


# Steam Data

In [12]:
import requests
import json
from itertools import islice

def get_top_played_games_steamspy(limit=10):
    url = 'https://steamspy.com/api.php?request=top100in2weeks'
    response = requests.get(url)
    data = response.json()
    top_games = dict(islice(data.items(), limit))
    return [game['appid'] for game in top_games.values()]

def get_game_data(appid):
    url = f'http://store.steampowered.com/api/appdetails?appids={appid}'
    response = requests.get(url)
    data = response.json()
    if data[str(appid)]['success']:
        return data[str(appid)]['data']
    else:
        return None

def print_game_info(game_data):
    if game_data:
        print(f"Title: {game_data.get('name', 'N/A')}")
        print(f"Release Date: {game_data.get('release_date', {}).get('date', 'N/A')}")
        if 'price_overview' in game_data:
            print(f"Price: {game_data['price_overview']['final_formatted']}")
        else:
            print("Price: Free")
        print(f"Metacritic Score: {game_data.get('metacritic', {}).get('score', 'N/A')}")
        print(f"Recommendations: {game_data.get('recommendations', {}).get('total', 'N/A')}")
        print(f"Developers: {', '.join(game_data.get('developers', []))}")
        print(f"Publishers: {', '.join(game_data.get('publishers', []))}")
    else:
        print("Game data not found.")

# Fetch the list of top played games from SteamSpy
top_played_games = get_top_played_games_steamspy(limit=200)
print(top_played_games)


[570, 730, 578080, 1623730, 1172470, 1063730, 2358720, 1938090, 271590, 550, 553850, 1599340, 304930, 236390, 1245620, 105600, 440, 291550, 1086940, 431960, 4000, 359550, 1085660, 252490, 346110, 892970, 1091500, 218620, 901583, 238960, 242760, 322330, 1203220, 899770, 413150, 340, 381210, 945360, 1097150, 292030, 227300, 444090, 552990, 438100, 320, 272060, 739630, 990080, 620, 227940, 70, 252950, 1966720, 1517290, 1240440, 582010, 240, 108600, 386360, 10, 648800, 1468810, 444200, 755790, 1174180, 250900, 550650, 367520, 301520, 239140, 477160, 251570, 814380, 1222670, 417910, 433850, 594650, 220, 230410, 219990, 261550, 8930, 304050, 107410, 377160, 532210, 255710, 1326470, 289070, 526870, 1238810, 60, 221100, 1089350, 204360, 632360, 96000, 275850, 1811260, 322170]


In [13]:
import numpy as np

# Fetch and store game details for each most played game
game_data_list = []

for appid in top_played_games:
    game_data = get_game_data(appid)
    if game_data:
        game_data_list.append(game_data)

# Convert the list of game data to a numpy array
game_data_array = np.array(game_data_list)

In [14]:
import json

# Pretty print the first game's data
print_game_info(game_data_array[0])

Title: Dota 2
Release Date: Jul 9, 2013
Price: Free
Metacritic Score: 90
Recommendations: 14342
Developers: Valve
Publishers: Valve


In [42]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool

# Convert price to numeric value
def convert_price(price):
    try:
        if price == 'Free':
            return 0
        else:
            return float(price.replace('$', ''))
    except ValueError:
        return None
    
    # Convert score to float, or return None
    def convert_score(score):
        try:
            return float(score)
        except (ValueError, TypeError):
            return None

# Extract game info
def get_game_info(game_data):
    title = game_data.get('name', 'N/A')
    price = game_data.get('price_overview', {}).get('final_formatted', 'Free')
    score = game_data.get('metacritic', {}).get('score', None)
    recommendations = game_data.get('recommendations', {}).get('total', 'N/A')
    return title, convert_price(price), score, recommendations


# Prepare data for plotting
plot_data = {
    'Title': [],
    'Price': [],
    'Metacritic_Score': [],
    'Recommendations': []
}

for game in game_data_array:
    title, price, score, recommendations = get_game_info(game)
    if price is not None and score is not None:
        plot_data['Title'].append(title)
        plot_data['Price'].append(price)
        plot_data['Metacritic_Score'].append(score)
        plot_data['Recommendations'].append(recommendations)

# Create ColumnDataSource
source = ColumnDataSource(data=plot_data)

# Create the plot
p = figure(title="Price vs Metacritic Score", x_axis_label='Price (USD)', y_axis_label='Metacritic Score', tools="pan,wheel_zoom,box_zoom,reset,hover,save")

# Add circle glyphs
p.circle('Price', 'Metacritic_Score', size=10, source=source, fill_alpha=0.6)

# Add hover tool
hover = p.select_one(HoverTool)
hover.tooltips = [
    ("Title", "@Title"),
    ("Price", "@Price"),
    ("Metacritic Score", "@Metacritic_Score"),
    ("Recommendations", "@Recommendations")
]
]

# Show the plot in the notebook
output_notebook()
show(p)

SyntaxError: unmatched ']' (3408437753.py, line 63)